## Trigram LM

In [1]:
names = open('/content/drive/MyDrive/LM/names.txt', 'r').read().splitlines()

To learn statistics, we can either use counts or a neural network. To get trigram counts:

In [2]:
counts = {}
for n in names:
  chs = ['.', '.']+list(n)+['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    trigram = (ch1, ch2, ch3)
    counts[trigram] = counts.get(trigram, 0) + 1

In [3]:
sorted(counts.items(), key=lambda kv: -kv[1])[:20]

[(('.', '.', 'a'), 4410),
 (('.', '.', 'k'), 2963),
 (('.', '.', 'm'), 2538),
 (('.', '.', 'j'), 2422),
 (('.', '.', 's'), 2055),
 (('a', 'h', '.'), 1714),
 (('.', '.', 'd'), 1690),
 (('n', 'a', '.'), 1673),
 (('.', '.', 'r'), 1639),
 (('.', '.', 'l'), 1572),
 (('.', '.', 'c'), 1542),
 (('.', '.', 'e'), 1531),
 (('a', 'n', '.'), 1509),
 (('o', 'n', '.'), 1503),
 (('.', 'm', 'a'), 1453),
 (('.', '.', 't'), 1308),
 (('.', '.', 'b'), 1306),
 (('.', 'j', 'a'), 1255),
 (('.', 'k', 'a'), 1254),
 (('e', 'n', '.'), 1217)]

In [4]:
chs = sorted(list(set(''.join(names))))
chs.append('.') # ATTENTION: Because it is appended to the end, index for . token will be 26
len(chs)

27

In [5]:
ch2i = {ch:i for i, ch in enumerate(chs)}
i2ch = {i:ch for ch, i in ch2i.items()}

In the trigram case, entry $N[i_{1},i_{2},i_{3}]$ is the number of times the character indexed by $i_{3}$ follows the one with index $i_{2}$ that is followed by the one indexed with $i_{1}$.

In [13]:
import torch
N = torch.zeros((27, 27, 27), dtype=torch.int32)
# "training the model"
for n in names:
  chs = ['.', '.']+list(n)+['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    i_1 = ch2i[ch1]
    i_2 = ch2i[ch2]
    i_3 = ch2i[ch3]
    N[i_1, i_2, i_3] += 1

In the trigram case, training the model means to estimate the probabilities $P(w_{i} | w_{i-1}, w_{i-2})$. Now we sample from this trigram character level language model, i.e. predict the next character given the previous two.

In [14]:
N[ch2i['.'], ch2i['.'], :]

tensor([4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963, 1572,
        2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,  134,
         535,  929,    0], dtype=torch.int32)

In [15]:
# turn raw counts into probabilities
p = N[ch2i['.'],ch2i['.'],:].float()
p /= p.sum() # normalize
p, p.sum() # p is now a probability distribution

(tensor([0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273, 0.0184,
         0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029, 0.0512,
         0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290, 0.0000]),
 tensor(1.))

In [16]:
ixs = torch.multinomial(p, num_samples=20, replacement=True) # you give me probs and I give you integers according to the probability distr
ixs

tensor([25, 13, 17,  0,  3,  1, 24,  4, 15, 14, 17, 25,  0, 11,  0,  2, 10,  0,
         4,  3])

Broadcasting: To sum over the third dimension of a 3D tensor, use `.sum(2, keepdim=True)`. `keepdim=True` keeps the dimension alog which the summation was made, we need this for broadcasting to work properly. `keepdim=False` would squueze out the dimension which sums are collapsed, resulting in a 2D tensor in our case, then by the broadcasting rules of PyTorch, these sums would be copied over the first axis instead of third.

In [17]:
P = N.float()
P /= P.sum(2, keepdim=True) # /= is an in-place op

In [18]:
# sampling from the trigram model
for i in range(10):
  ix = ch2i['.']
  ix1 = ch2i['.']
  out = [i2ch[ix], i2ch[ix1]]
  while True:
    p = P[ix,ix1]
    ix2 = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(i2ch[ix2])
    if ix2 == ch2i['.']:
      break
    else:
      ix = ix1
      ix1 = ix2
  print(''.join(out))

..hiahraleig.
..ca.
..eatleild.
..nuaazey.
..khaleilla.
..die.
..abechemi.
..marpen.
..alinettenne.
..tevise.


This model we trained decreases the entropy, just like the living. The results look like this with no training:

In [19]:
for i in range(10):
  ix = ch2i['.']
  ix1 = ch2i['.']
  out = [i2ch[ix],i2ch[ix1]]
  while True:
    p_even = torch.ones(27) / 27.0
    ix2 = torch.multinomial(p, num_samples=1, replacement=True).item()
    out.append(i2ch[ix2])
    if ix2 == ch2i['.']:
      break
    else:
      ix = ix1
      ix1 = ix2
  print(''.join(out))

...
..hl.
..yli.
..nrm.
..r.
...
..nnnoamrhn.
..vn.
...
...


In [20]:
log_likelihood = 0.0
k = 0
for n in names:
  chs = ['.', '.']+list(n)+['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    i_1 = ch2i[ch1]
    i_2 = ch2i[ch2]
    i_3 = ch2i[ch3]
    prob = P[i_1,i_2,i_3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    k += 1
    # print(f'{ch1}{ch2}{ch3}: {prob:.4f} {logprob:.4f}')
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}') # lower the better, a convenient loss metric
print(f'{nll/k}') # average nll

log_likelihood=tensor(-498647.7812)
nll=tensor(498647.7812)
2.185652017593384


As we can observe from this metric, trigram lm is an improvement over the bigram lm.

### Neural network approach

Instead of counting, we will learn the counts array by optimizing a set of parameters.

In [21]:
# creating the trigram dataset for training
xs, ys = [], []
for n in names:
  chs = ['.', '.']+list(n)+['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    i_1 = ch2i[ch1]
    i_2 = ch2i[ch2]
    i_3 = ch2i[ch3]
    xs.append([i_1, i_2])
    ys.append(i_3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [22]:
xs.shape

torch.Size([228146, 2])

In [23]:
import torch.nn.functional as F

In [24]:
F.one_hot(xs, num_classes=27).shape

torch.Size([228146, 2, 27])

In [25]:
W = torch.randn((54,27), requires_grad=True)

Apply `W.view((27,27,27)` after training to keep the analogy between N and W.

In [29]:
# gradient descent
for k in range(30):

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float().view((xs.shape[0],54))
  ## softmax: takes in the logits, exponentiates them and normalizes
  logits = xenc @ W # log-counts
  counts = logits.exp() # equivalent to N
  p = counts / counts.sum(1, keepdim=True)
  loss = -p[torch.arange(xs.shape[0]), ys].log().mean() # average nll
  print(loss.item())

  # backward pass
  W.grad = None
  loss.backward()
  W.grad.shape # influences of weights on the loss function

  # gradient update
  W.data += -10 * W.grad

2.40067458152771
2.384706497192383
2.3812673091888428
2.3799402713775635
2.379206895828247
2.378702163696289
2.378312110900879
2.377990961074829
2.377715826034546
2.3774731159210205
2.377254009246826
2.377051830291748
2.376863479614258
2.3766846656799316
2.376514196395874
2.376349687576294
2.376190662384033
2.3760361671447754
2.375885009765625
2.375736713409424
2.37559175491333
2.37544846534729
2.375307559967041
2.375168561935425
2.3750312328338623
2.3748953342437744
2.374760627746582
2.3746275901794434
2.3744957447052
2.3743646144866943


In [30]:
for i in range(5):
  print(f'trigram:{i2ch[xs[i][0].item()],i2ch[xs[i][1].item()],i2ch[ys[i].item()]}')
  print(f'output probability distribution given the input:{p[i]}')
  print(f'probability assigned to the actual label:{p[i,ys[i]]}')
  print(f'log likelihood:{torch.log(p[i,ys[i]])}')
  print(f'nll:{-torch.log(p[i,ys[i]])}')

trigram:('.', '.', 'e')
output probability distribution given the input:tensor([0.1358, 0.0402, 0.0479, 0.0528, 0.0480, 0.0127, 0.0204, 0.0272, 0.0189,
        0.0748, 0.0918, 0.0491, 0.0793, 0.0356, 0.0129, 0.0155, 0.0031, 0.0511,
        0.0640, 0.0399, 0.0047, 0.0131, 0.0102, 0.0041, 0.0170, 0.0297, 0.0003],
       grad_fn=<SelectBackward0>)
probability assigned to the actual label:0.04803650826215744
log likelihood:-3.0357940196990967
nll:3.0357940196990967
trigram:('.', 'e', 'm')
output probability distribution given the input:tensor([0.0798, 0.0071, 0.0040, 0.0283, 0.0720, 0.0020, 0.0026, 0.0108, 0.0402,
        0.0021, 0.0053, 0.2404, 0.0641, 0.0664, 0.0294, 0.0035, 0.0003, 0.1670,
        0.0366, 0.0190, 0.0159, 0.0335, 0.0016, 0.0019, 0.0521, 0.0115, 0.0026],
       grad_fn=<SelectBackward0>)
probability assigned to the actual label:0.06408719718456268
log likelihood:-2.7475106716156006
nll:2.7475106716156006
trigram:('e', 'm', 'm')
output probability distribution given the in

A good model assigns high probabilities to data in the training set. 